# 🔍 Persian OCR - استخراج متن فارسی

## 🚀 راهنمای استفاده:
1. همه سل‌ها رو از بالا به پایین اجرا کن
2. فایل PDF رو آپلود کن 
3. منتظر باش تا متن استخراج بشه
4. فایل نتیجه رو دانلود کن

---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MohammadHNdev/persian-ocr-simple/blob/main/Persian_OCR_Simple.ipynb)

In [ ]:
# 🔧 نصب و راه‌اندازی (فقط یکبار اجرا کن)
print("🚀 در حال نصب ابزارهای مورد نیاز...")

# نصب کتابخانه‌ها
!pip install pytesseract pdf2image Pillow opencv-python-headless tqdm > /dev/null 2>&1

# نصب Tesseract
!sudo apt update > /dev/null 2>&1
!sudo apt install -y tesseract-ocr tesseract-ocr-fas poppler-utils > /dev/null 2>&1

print("✅ نصب کامل شد! حالا می‌تونی ادامه بدی.")

In [ ]:
# 📚 وارد کردن کتابخانه‌ها
import pytesseract
from PIL import Image, ImageEnhance
from pdf2image import convert_from_path
import os
import time
from tqdm.notebook import tqdm
import concurrent.futures
from google.colab import files
import shutil

print("📚 کتابخانه‌ها آماده شدند!")

In [ ]:
# 🧠 تابع اصلی OCR
def extract_text_from_pdf(pdf_path, dpi=350, max_workers=3):
    """استخراج متن فارسی از PDF"""
    
    print(f"📄 شروع پردازش: {os.path.basename(pdf_path)}")
    start_time = time.time()
    
    # پوشه موقت
    temp_dir = '/tmp/ocr_images'
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)
    
    try:
        # تبدیل PDF به تصاویر
        print("🖼️ تبدیل PDF به تصاویر...")
        images = convert_from_path(pdf_path, dpi=dpi, fmt='jpeg')
        
        print(f"📊 {len(images)} صفحه یافت شد")
        
        # ذخیره تصاویر
        image_paths = []
        for i, img in enumerate(images):
            img_path = os.path.join(temp_dir, f"page_{i+1:03d}.jpg")
            img.save(img_path, 'JPEG', quality=95)
            image_paths.append((img_path, i+1))
        
        # پاکسازی حافظه
        for img in images:
            del img
        del images
        
        # OCR موازی
        print("🔍 شروع OCR...")
        results = []
        
        def process_page(img_info):
            img_path, page_num = img_info
            try:
                # بارگذاری و بهبود تصویر
                img = Image.open(img_path)
                img_gray = img.convert('L')
                
                # افزایش کنتراست
                enhancer = ImageEnhance.Contrast(img_gray)
                img_enhanced = enhancer.enhance(2.0)
                
                # OCR
                text = pytesseract.image_to_string(
                    img_enhanced, 
                    lang='fas',
                    config='--psm 3'
                )
                
                # پاکسازی
                del img, img_gray, img_enhanced
                os.remove(img_path)
                
                return {
                    'page': page_num,
                    'text': text.strip(),
                    'chars': len(text.strip()),
                    'success': True
                }
            except Exception as e:
                return {
                    'page': page_num,
                    'text': f"خطا: {e}",
                    'chars': 0,
                    'success': False
                }
        
        # پردازش موازی
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(tqdm(
                executor.map(process_page, image_paths),
                total=len(image_paths),
                desc="پردازش صفحات"
            ))
        
        # مرتب‌سازی بر اساس شماره صفحه
        results.sort(key=lambda x: x['page'])
        
        # ساخت فایل نهایی
        output_filename = f"extracted_text_{int(time.time())}.txt"
        
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(f"# متن استخراج شده از: {os.path.basename(pdf_path)}\n")
            f.write(f"# تاریخ: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("\n" + "="*60 + "\n\n")
            
            for result in results:
                f.write(f"\n--- صفحه {result['page']} ---\n")
                f.write(result['text'])
                f.write("\n\n")
        
        # آمار نهایی
        successful_pages = sum(1 for r in results if r['success'])
        total_chars = sum(r['chars'] for r in results if r['success'])
        processing_time = time.time() - start_time
        
        print(f"\n🎉 پردازش کامل شد!")
        print(f"📊 صفحات موفق: {successful_pages}/{len(results)}")
        print(f"📝 کل کاراکترها: {total_chars:,}")
        print(f"⏱️ زمان: {processing_time:.1f} ثانیه")
        print(f"📁 فایل: {output_filename}")
        
        return {
            'success': True,
            'output_file': output_filename,
            'stats': {
                'total_pages': len(results),
                'successful_pages': successful_pages,
                'total_characters': total_chars,
                'processing_time': processing_time
            },
            'preview': results[0]['text'][:500] if results and results[0]['success'] else ""
        }
        
    except Exception as e:
        print(f"❌ خطا: {e}")
        return {'success': False, 'error': str(e)}
        
    finally:
        # پاکسازی
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

print("🧠 تابع OCR آماده شد!")

In [ ]:
# 📤 آپلود فایل PDF
print("📤 لطفاً فایل PDF خود را انتخاب کنید:")
print("(فقط فایل‌های PDF قابل قبول هستند)")

uploaded = files.upload()

if uploaded:
    # انتخاب اولین فایل PDF
    pdf_file = None
    for filename in uploaded.keys():
        if filename.lower().endswith('.pdf'):
            pdf_file = filename
            break
    
    if pdf_file:
        file_size = len(uploaded[pdf_file]) / (1024*1024)  # MB
        print(f"✅ فایل آپلود شد: {pdf_file} ({file_size:.1f} MB)")
        
        if file_size > 50:
            print("⚠️ فایل بزرگه! ممکنه کمی طول بکشه...")
    else:
        print("❌ فایل PDF یافت نشد! لطفاً فقط فایل PDF آپلود کنید.")
        pdf_file = None
else:
    print("❌ هیچ فایلی آپلود نشد!")
    pdf_file = None

In [ ]:
# 🚀 شروع OCR
if 'pdf_file' in locals() and pdf_file:
    print(f"🔥 شروع پردازش {pdf_file}...")
    print("☕ یکم صبر کن، داره کار می‌کنه...")
    
    # پردازش
    result = extract_text_from_pdf(pdf_file)
    
    if result['success']:
        print("\n" + "="*50)
        print("🎊 تبریک! OCR با موفقیت انجام شد!")
        print("="*50)
        
        stats = result['stats']
        print(f"📄 صفحات پردازش شده: {stats['successful_pages']}/{stats['total_pages']}")
        print(f"📝 کل کاراکترها: {stats['total_characters']:,}")
        print(f"⏱️ زمان پردازش: {stats['processing_time']:.1f} ثانیه")
        print(f"⚡ سرعت: {stats['total_characters']/stats['processing_time']:.0f} کاراکتر/ثانیه")
        
        # نمایش نمونه متن
        if result['preview']:
            print(f"\n📖 نمونه متن استخراج شده:")
            print("-"*40)
            print(result['preview'])
            if len(result['preview']) >= 500:
                print("...")
        
        # دانلود فایل
        print(f"\n💾 آماده‌سازی دانلود...")
        try:
            files.download(result['output_file'])
            print(f"✅ فایل {result['output_file']} دانلود شد!")
        except:
            print(f"⚠️ فایل در پنل فایل‌های Colab موجود است: {result['output_file']}")
            
    else:
        print(f"❌ متأسفانه خطایی رخ داد: {result.get('error', 'نامشخص')}")
        print("💡 می‌تونی دوباره امتحان کنی یا فایل کوچک‌تری انتخاب کنی.")
        
else:
    print("❌ ابتدا باید فایل PDF آپلود کنی!")
    print("👆 به سل قبلی برگرد و فایل رو آپلود کن.")

---

## 🎉 تمام شد!

### 📋 اگر مشکلی داشتی:
- **فایل خیلی بزرگه؟** سعی کن فایل کوچک‌تری انتخاب کنی
- **خطا گرفتی؟** همه سل‌ها رو از اول اجرا کن
- **متن خالی شد؟** ممکنه کیفیت PDF پایین باشه

### 💡 نکات:
- برای نتیجه بهتر، PDF با کیفیت بالا استفاده کن
- فایل‌های زیر 20 صفحه سریع‌تر پردازش می‌شن
- اگر runtime قطع شد، از اول شروع کن

---

**ساخته شده با ❤️ برای OCR فارسی**